# HEALTH DATA 

## 1. Import Packages

In [6]:
import sqlite3
import random
import string
from datetime import datetime, timedelta
import uuid

## 2. Safely close previous connections.

In [22]:
try:
    conn.close()
except:
    pass

## 3. Connect to data base.

In [25]:
conn = sqlite3.connect("health_data.db")
cur = conn.cursor()

In [27]:
# Enable FK support
cur.execute("PRAGMA foreign_keys = ON;")

## 4. Drop Old Tables if exists.

In [30]:
cur.executescript("""
DROP TABLE IF EXISTS Person;
DROP TABLE IF EXISTS HealthRecord;
DROP TABLE IF EXISTS EmploymentRecord;
""")

## 5. Create Tables.

In [33]:
# TABLE 1: Client (Primary Key: client_id)

cur.execute("""
CREATE TABLE Person (
    person_id TEXT PRIMARY KEY,
    full_name TEXT,
    gender TEXT CHECK(gender IN ('Male','Female')),
    age INTEGER CHECK(age >= 0),
    height REAL,
    weight REAL,
    bmi REAL,
    diabetes TEXT CHECK(diabetes IN ('Yes','No')),
    employee_status TEXT CHECK(employee_status IN ('Employed','Unemployed','Retired'))
);
""")

In [35]:
# TABLE 2: HEALTH RECORD  (Composite Key)
# Composite Primary Key: (record_id, person_id)
# record_id = random alphanumeric ID

cur.execute("""
CREATE TABLE HealthRecord (
    record_id TEXT,
    person_id TEXT,
    calories INTEGER,
    blood_pressure TEXT,
    checkup_date TEXT,
    PRIMARY KEY (record_id, person_id),
    FOREIGN KEY (person_id) REFERENCES Person(person_id)
);
""")

In [37]:
# TABLE 3: EMPLOYMENT RECORD
# Primary Key = emp_record_id (random)
# Foreign Key = person_id

cur.execute("""
CREATE TABLE EmploymentRecord (
    emp_record_id TEXT PRIMARY KEY,
    person_id TEXT,
    employer TEXT,
    years_exp INTEGER,
    FOREIGN KEY(person_id) REFERENCES Person(person_id)
);
""")

In [39]:
# FUNCTIONS FOR RANDOM REALISTIC IDs

def gen_org_id():
    """Generate realistic organizational-style person ID."""
    return "P-" + "".join(random.choices(string.ascii_uppercase + string.digits, k=7))

def gen_record_id(prefix):
    return prefix + "-" + "".join(random.choices(string.ascii_lowercase + string.digits, k=10))

def calculate_bmi(w, h):
    if not h or not w:
        return None
    h_m = h / 100
    return round(w / (h_m * h_m), 2)

## 6.Generate Person Data

In [42]:
genders = ["Male", "Female"]
employees = ["Employed", "Unemployed", "Retired"]
diabetes_options = ["Yes", "No"]

persons = []
used_ids = set()

for _ in range(1000):
    
    # Realistic random IDs (no auto-increment)
    p_id = gen_org_id()
    while p_id in used_ids:  # ensure unique
        p_id = gen_org_id()
    used_ids.add(p_id)

    full_name = random.choice(["Alex", "John", "Mary", "Susan", "Mark", "Nina", "Paul", "Mia"]) + " " + \
                random.choice(["Smith", "Doe", "Miles", "Brown", "Hill", "Adams"])

    gender = random.choice(genders)
    age = random.randint(20, 80)

    # Introduce missing values 5% height, 5% weight
    height = round(random.uniform(150, 200), 1) if random.random() > 0.05 else None
    weight = round(random.uniform(50, 120), 1) if random.random() > 0.05 else None

    bmi = calculate_bmi(weight, height)

    # 3% missing diabetes values
    diabetes = random.choice(diabetes_options) if random.random() > 0.03 else None

    employee_status = random.choice(employees)

    # 2% duplicate rows (duplicate full_name, height, weight)
    if random.random() < 0.02:
        height = height
        weight = weight

    persons.append((p_id, full_name, gender, age, height, weight, bmi, diabetes, employee_status))

cur.executemany("""
INSERT INTO Person (person_id, full_name, gender, age, height, weight, bmi, diabetes, employee_status)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""", persons)

## 7. GENERATE HEALTH RECORDS

In [46]:
# Composite key: (record_id, person_id)

cur.execute("SELECT person_id FROM Person;")
all_persons = [x[0] for x in cur.fetchall()]

health_records = []

for pid in all_persons:
    for _ in range(random.randint(1, 3)):
        rid = gen_record_id("HR")

        calories = random.randint(1500, 3500)
        systolic = random.randint(100, 160)
        diastolic = random.randint(60, 100)
        bp = f"{systolic}/{diastolic}"

        days_ago = random.randint(10, 1500)
        date = (datetime.now() - timedelta(days=days_ago)).strftime("%Y-%m-%d")

        health_records.append((rid, pid, calories, bp, date))

cur.executemany("""
INSERT INTO HealthRecord (record_id, person_id, calories, blood_pressure, checkup_date)
VALUES (?, ?, ?, ?, ?)
""", health_records)

## 8. GENERATE EMPLOYMENT RECORDS

In [49]:
employment_records = []
companies = ["TechCorp", "MediLife", "EduWorks", "AgriFoods", "RetailCo", "TranspoX"]

for pid in all_persons:
    if random.random() < 0.80:
        emp_id = gen_record_id("EMP")
        employer = random.choice(companies)
        years = random.randint(1, 40)

        # 3% missing employer/years
        if random.random() < 0.03:
            employer = None
            years = None

        employment_records.append((emp_id, pid, employer, years))

cur.executemany("""
INSERT INTO EmploymentRecord (emp_record_id, person_id, employer, years_exp)
VALUES (?, ?, ?, ?)
""", employment_records)

## 8. Commit and Close.

In [52]:
conn.commit()
conn.close()

In [11]:
# Download all tables as CSV files.

import sqlite3
import pandas as pd
import os

# Connect to your database
db_path = "health_data.db"
conn = sqlite3.connect(db_path)

# Create export folder (optional)
export_folder = "exported_tables"
os.makedirs(export_folder, exist_ok=True)

# Get all table names
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)

print("Tables found:", tables["name"].tolist())

# Export each table as CSV
for table in tables["name"]:
    df = pd.read_sql_query(f"SELECT * FROM {table};", conn)
    csv_path = os.path.join(export_folder, f"{table}.csv")
    df.to_csv(csv_path, index=False)

conn.close()



Tables found: ['sqlite_sequence', 'Person', 'HealthRecord', 'EmploymentRecord']
